In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import zipfile
'''
Предварительно на гугл-диск нужно закинуть архив с данными imdb:
https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
'''
z = zipfile.ZipFile('/content/drive/My Drive/imdb-dataset-of-50k-movie-reviews.zip', 'r')
z.extractall()

In [0]:
!ls

 drive	'IMDB Dataset.csv'   sample_data


In [0]:
import pandas as pd
data_imdb = pd.read_csv('IMDB Dataset.csv')

In [0]:
data_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
data_imdb['review'].iloc[3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [0]:
# OHE для таргетовой переменной
map_sentiment = {'negative': 0,'positive': 1}
data_imdb['sentiment'] = data_imdb['sentiment'].map(map_sentiment)

In [0]:
data_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [0]:
# Пример грязного текста!
data_imdb['review'][5]

'Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.'

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# Произведем очистку текста!

import re

stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

from nltk.corpus import stopwords
 
stopWords = set(stopwords.words('english'))

def clean_text(text, stop_words):

  text = text.lower()                 # привеле к нижнему регистру
  reg = re.compile('[^a-zA-Z0-9 ]')   # создали регулярное выражение
  text = reg.sub('', text)
  tokens = text.split(' ')
  clean_text = ''
  for token in tokens:
    if ( token in stop_words ) | ( len(token) < 3 ):
      pass
    else:
      clean_text+= ' '+token

  return clean_text

data_imdb['review'] = data_imdb['review'].apply(lambda x: clean_text(x, stop_words))

In [0]:
# Пример чистого текста!
data_imdb['review'][5]

' probably alltime favorite movie story selflessness sacrifice dedication noble cause preachy boring never gets old despite seen times last years paul lukas performance brings tears eyes bette davis one truly sympathetic roles delight kids grandma says like dressedup midgets children makes fun watch mothers slow awakening whats happening world roof believable startling dozen thumbs theyd movie'

In [0]:
 # используем мешок слов для кодирования текстовых данных
 from sklearn.feature_extraction.text import CountVectorizer
 vectorizer = CountVectorizer()
 data_BOW = vectorizer.fit_transform(data_imdb['review'])

In [0]:
print(type(data_BOW),data_BOW.shape)

<class 'scipy.sparse.csr.csr_matrix'> (50000, 180641)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_BOW, data_imdb['sentiment'], test_size=0.25, random_state=42)

In [0]:
print(X_train.shape, X_test.shape)

(37500, 180641) (12500, 180641)


In [0]:
 # обучаем логистическую регрессию
 from sklearn.linear_model import LogisticRegression
 clf = LogisticRegression(random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
y_predict = clf.predict(X_test)

In [0]:
# неплохой результат!
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.88616

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data_TFIDF = vectorizer.fit_transform(data_imdb['review'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_TFIDF, data_imdb['sentiment'], test_size=0.25, random_state=42)

In [0]:
 clf = LogisticRegression(random_state=42).fit(X_train, y_train)

In [0]:
y_predict = clf.predict(X_test)

In [0]:
accuracy_score(y_test, y_predict)

0.89864